In [9]:
import eventregistry
import pandas as pd
from eventregistry import *
import time, datetime
import json
import os
from pandas import json_normalize

er = EventRegistry(apiKey = "671b8422-f66f-4b07-b1b3-4af12364ec07")
import openai

In [2]:
starttime = time.time()
updatesAfterTm = None
# while True:
for i in range(10):
    # q = QueryArticles(keywords="strike", keywordsLoc="title")
    q = QueryArticles (keywords = QueryItems.OR(["Trump", "Harris"]),
                      lang="eng")
                      #keywordOper="Trump" or "Harris")
    q.setRequestedResult(
        RequestArticlesRecentActivity(
            # download at most 2000 articles. if less of matching articles were added in last 10 minutes, less will be returned
            maxArticleCount=10,
            # specify the last time that was used when making the request - only articles added after that will be potentially returned
            updatesAfterTm = updatesAfterTm
        ))

    res = er.execQuery(q)
 


In [12]:

json_folder = "C:\\Users\\Mingrun_Sun2\\Documents\\GitHub\\ElectionGPT\\NLP\\"
file_name = os.path.join(json_folder, 'output.json')  # Combine folder and file name

# Assuming 'res' is your JSON data
with open(file_name, 'w') as json_file:
    json.dump(res, json_file, indent=4)




FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Mingrun_Sun2\\Documents\\GitHub\\ElectionGPT\\NLP\\/output.json'

In [ ]:

# Loop over each file and append the data to the list
with open("C:\\Users\\Mingrun_Sun2\\Documents\\GitHub\\ElectionGPT\\NLP\\output.json", 'r') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))
# Extract the relevant part ('activity' in your case)
results = data.get('recentActivityArticles', {}).get('activity', [])

In [ ]:
df = json_normalize(results)
df = df.applymap(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

df.to_csv('full_news_data.csv', index=False)

In [3]:
# fine tuning start

with open("C:\\Users\\Mingrun_Sun2\\Documents\\GitHub\\ElectionGPT\\NLP\\output.json", 'r') as f:
    news_data = json.load(f)

# Prepare the data for fine-tuning
jsonl_data = []

for article in news_data['recentActivityArticles']['activity']:
    title = article.get('title', '')
    body = article.get('body', '')
    
    # Create a prompt using title and body of the article
    prompt = f"Summarize the following article: {title}. Body: {body}"

    # Placeholder for completion (manual or automated summarization)
    completion = "Insert summary or story here"  # You can automate or manually generate this part
    
    jsonl_data.append({
        "prompt": prompt,
        "completion": completion
    })

# Save the reformatted data to a .jsonl file
with open('news_data_for_fine_tuning.jsonl', 'w') as outfile:
    for entry in jsonl_data:
        json.dump(entry, outfile)
        outfile.write('\n')

print("Data has been successfully reformatted and saved as news_data_for_fine_tuning.jsonl")

Data has been successfully reformatted and saved as news_data_for_fine_tuning.jsonl


In [11]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-None-nZqg5cC5FVP1Oq5pZjIkT3BlbkFJlhWzmMlVDlqoaxd5uVCd"))


ImportError: cannot import name 'OpenAI' from 'openai' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openai/__init__.py)

In [5]:
client.files.create(
  file=open("news_data_for_fine_tuning.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-cJl5XoE4UiWNhLb1ttjhptFw', bytes=42259, created_at=1725927449, filename='news_data_for_fine_tuning.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [6]:
# Function to generate completions using GPT-3.5
def generate_completion(article_text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        prompt=f"Read the following article: {article_text}",
        max_tokens=1000  # Adjust max_tokens as needed
    )
    return response['choices'][0]['text']



In [ ]:
# Load your news dataset (assuming a CSV or Excel file)
df = pd.read_csv('news_articles.csv')  # Replace with your actual data file

# Prepare a list to store the JSONL data
jsonl_data = []

# Loop through each news article and generate a completion
for index, row in df.iterrows():
    article_text = row['News Article']  # Adjust based on your CSV structure
    prompt = f"Read the following article: {article_text}"
    completion = generate_completion(article_text)
    
    # Append the JSON object to the list
    jsonl_data.append({
        "prompt": prompt,
        "completion": completion
    })

# Save the data to a JSONL file
with open('news_data_for_fine_tuning.jsonl', 'w') as outfile:
    for entry in jsonl_data:
        json.dump(entry, outfile)
        outfile.write('\n')

print("JSONL file created successfully!")

In [62]:
client.fine_tuning.jobs.create(
  training_file="file-TrXUVTsApEB4j7WaAtGXF0V4", 
  model="gpt-4o-2024-08-06"
)

FineTuningJob(id='ftjob-b474T6zYSwLJCiGQkU29nxyg', created_at=1725920312, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-UnkNAXSFjByrON8hgvdjarxx', result_files=[], seed=1198012277, status='validating_files', trained_tokens=None, training_file='file-TrXUVTsApEB4j7WaAtGXF0V4', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
client.fine_tuning.jobs.retrieve('ftjob-b474T6zYSwLJCiGQkU29nxyg')
# Retrieve fine-tuning logs


NotFoundError: Error code: 404 - {'error': {'message': "The model 'ftjob-b474T6zYSwLJCiGQkU29nxyg' does not exist", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}